# 策略

找到30天内涨幅在80%以上的牛股，当其日线rsi回到10%以下低位时，发出买入信号

In [1]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
await init_notebook(use_omicron=True)

In [21]:
async def scan(tm, adv=0.6, wr = 0.618, ddv = 0.01, n=30, df=True):    
    end = arrow.get(tm).date()
    start = tf.day_shift(end, -n+1)
    
    results = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)
            bars = bars[np.isfinite(bars["close"])]
            
            close = bars["close"]
            
            if len(bars) < n * 0.9:
                continue

            adv_ = np.max(close)/np.min(close) - 1

            if np.argmin(close) >= np.argmax(close) or adv_ < adv:
                continue
                
            wr_ = close[-1] / np.max(close)
            ddv_ = moving_average(close[1:] / close[:-1] - 1, 5)[-1]
            rsi = relative_strength_index(close, 6)
            
            min_rsi = np.min(rsi[-4:])
            prsi = rsiday.get_proba(code, min_rsi)
            
            results.append([sec.display_name, code, end, adv_, wr_, ddv_, prsi])
        except Exception:
            continue
        
    if df:
        return pd.DataFrame(results, columns=["name", "code", "date", "adv", "wr", "ddv", "prsi"])
    else:
        return results

In [22]:
df = await scan("2021-10-25")
df

,name,code,date,adv,wr,ddv,prsi
0,万科A,000002.XSHE,2021-10-25,0.186646,0.912273,0.003212,0.13801
1,美丽生态,000010.XSHE,2021-10-25,0.147945,0.902148,-0.008195,0.24179
2,深粮控股,000019.XSHE,2021-10-25,0.069016,0.968407,-0.002464,0.30606
3,京基智农,000048.XSHE,2021-10-25,0.063079,0.989657,0.007891,0.60083
4,德赛电池,000049.XSHE,2021-10-25,0.360666,1.000000,0.049242,0.95942
...,...,...,...,...,...,...,...
1055,野马电池,605378.XSHG,2021-10-25,0.199706,0.957082,-0.001273,NaN
1056,均瑶健康,605388.XSHG,2021-10-25,0.173463,0.956950,-0.001067,NaN
1057,晨光新材,605399.XSHG,2021-10-25,0.451005,1.000000,0.058550,NaN
1058,东鹏饮料,605499.XSHG,2021-10-25,0.219091,0.863208,-0.013549,NaN


In [29]:
df[(df.adv>0.6) & (df.ddv>0.02) & (df.wr < 0.8)]

,name,code,date,adv,wr,ddv,prsi
74,天音控股,000829.XSHE,2021-10-25,0.822045,0.784621,0.051522,0.90708
144,红宝丽,002165.XSHE,2021-10-25,0.878727,0.762360,0.024514,0.37134
147,云海金属,002182.XSHE,2021-10-25,0.632794,0.767327,0.037746,0.64752
368,奥克股份,300082.XSHE,2021-10-25,0.816353,0.752769,0.022236,0.41950
613,皖维高新,600063.XSHG,2021-10-25,0.625210,0.759049,0.037557,0.25332
937,金石资源,603505.XSHG,2021-10-25,0.622685,0.782454,0.034848,0.62512
